<a href="https://colab.research.google.com/github/Jrodrigo140787/DataSet/blob/master/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RandomizedSearchCV

#Carga de DataSet

In [ ]:
#Carga del Archivo Student-Mat
ruta ='/content/student-mat.csv'
Datos=pd.read_csv(ruta,sep=',')

# Actualizacion - nombre de columnas

Columnas= ['Escuela','Sexo','Edad','Direccion','Tamanio_F','Cohabitacion_P','Educacion_M','Educacion_P','Trabajo_M','Trabajo_P',
           'Razon','Tutor','Tiempo_V','Tiempo_E','Fallas','Apoyo_EA','Apoyo_EF','Pago_E','Actividad_E','Guarderia','Estudio_S',
           'Internet','Relacion_R','Relacion_F','Tiempo_L','Salida_A','Consumo_A_T','Consumo_A_FS','Estado_S','Ausencias_E',
           'Nota_1','Nota_2','Nota_F']
Datos.columns =Columnas
Datos.head()

,Escuela,Sexo,Edad,Direccion,Tamanio_F,Cohabitacion_P,Educacion_M,Educacion_P,Trabajo_M,Trabajo_P,Razon,Tutor,Tiempo_V,Tiempo_E,Fallas,Apoyo_EA,Apoyo_EF,Pago_E,Actividad_E,Guarderia,Estudio_S,Internet,Relacion_R,Relacion_F,Tiempo_L,Salida_A,Consumo_A_T,Consumo_A_FS,Estado_S,Ausencias_E,Nota_1,Nota_2,Nota_F
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


# Limpieza y Transformacion de Datos

validar la existencia de datos Nulos

In [64]:
# Columnas 
(Datos.isnull().sum()).sort_values(ascending=True)
#Filas
(Datos.isnull().sum(axis=1)).sort_values(ascending=True)

Datos.info()
# Varibles Cuantitativos y Cualitativos

var_cuant=['Edad','Educacion_M','Educacion_P','Tiempo_V', 'Tiempo_E', 'Fallas', 'Ausencias_E','Relacion_F','Tiempo_L','Salida_A',
         'Consumo_A_T','Consumo_A_FS','Estado_S', 'Nota_1', 'Nota_2']
Datos[var_cuant].head()

var_cat=['Escuela','Sexo','Direccion','Tamanio_F','Cohabitacion_P','Trabajo_M','Trabajo_P','Razon','Tutor','Apoyo_EA','Apoyo_EF',
         'Pago_E','Actividad_E','Guarderia','Estudio_S','Internet','Relacion_R']
Datos[var_cat].head()

#plt.scatter(Datos.Nota_2,Datos.Nota_1)

Variables - Categoricas

In [53]:
Datos[var_cat]=Datos[var_cat].astype('category')

Correlacion de variables cuantitativas

In [74]:
Datos[var_cuant].corr().style.background_gradient(cmap='coolwarm').set_precision(4)


,Edad,Educacion_M,Educacion_P,Tiempo_V,Tiempo_E,Fallas,Ausencias_E,Relacion_F,Tiempo_L,Salida_A,Consumo_A_T,Consumo_A_FS,Estado_S,Nota_1,Nota_2
Edad,1.0000,-0.1637,-0.1634,0.0706,-0.0041,0.2437,0.1752,0.0539,0.0164,0.1270,0.1311,0.1173,-0.0622,-0.0641,-0.1435
Educacion_M,-0.1637,1.0000,0.6235,-0.1716,0.0649,-0.2367,0.1003,-0.0039,0.0309,0.0641,0.0198,-0.0471,-0.0469,0.2053,0.2155
Educacion_P,-0.1634,0.6235,1.0000,-0.1582,-0.0092,-0.2504,0.0245,-0.0014,-0.0128,0.0431,0.0024,-0.0126,0.0147,0.1903,0.1649
Tiempo_V,0.0706,-0.1716,-0.1582,1.0000,-0.1009,0.0922,-0.0129,-0.0168,-0.0170,0.0285,0.1383,0.1341,0.0075,-0.0930,-0.1532
Tiempo_E,-0.0041,0.0649,-0.0092,-0.1009,1.0000,-0.1736,-0.0627,0.0397,-0.1432,-0.0639,-0.1960,-0.2538,-0.0756,0.1606,0.1359
Fallas,0.2437,-0.2367,-0.2504,0.0922,-0.1736,1.0000,0.0637,-0.0443,0.0920,0.1246,0.1360,0.1420,0.0658,-0.3547,-0.3559
Ausencias_E,0.1752,0.1003,0.0245,-0.0129,-0.0627,0.0637,1.0000,-0.0444,-0.0581,0.0443,0.1119,0.1363,-0.0299,-0.0310,-0.0318
Relacion_F,0.0539,-0.0039,-0.0014,-0.0168,0.0397,-0.0443,-0.0444,1.0000,0.1507,0.0646,-0.0776,-0.1134,0.0941,0.0222,-0.0183
Tiempo_L,0.0164,0.0309,-0.0128,-0.0170,-0.1432,0.0920,-0.0581,0.1507,1.0000,0.2850,0.2090,0.1478,0.0757,0.0126,-0.0138
Salida_A,0.1270,0.0641,0.0431,0.0285,-0.0639,0.1246,0.0443,0.0646,0.2850,1.0000,0.2670,0.4204,-0.0096,-0.1491,-0.1623


variables dicotómicas 

In [63]:
Dicos=pd.get_dummies(Datos[var_cat],drop_first = True)
Dicos.head()

,Escuela_MS,Sexo_M,Direccion_U,Tamanio_F_LE3,Cohabitacion_P_T,Trabajo_M_health,Trabajo_M_other,Trabajo_M_services,Trabajo_M_teacher,Trabajo_P_health,Trabajo_P_other,Trabajo_P_services,Trabajo_P_teacher,Razon_home,Razon_other,Razon_reputation,Tutor_mother,Tutor_other,Apoyo_EA_yes,Apoyo_EF_yes,Pago_E_yes,Actividad_E_yes,Guarderia_yes,Estudio_S_yes,Internet_yes,Relacion_R_yes
0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0
1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
2,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,0
3,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,1,1,1
4,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0


# Regresión

Variable Dependiente e Independiente

In [93]:
#Variable Dependiente e Independiente
X = pd.concat([Dicos, Datos[var_cuant]], axis=1)
Y= Datos.Nota_F

#Estimación por MCO
reg = LinearRegression()
modelo1 = reg.fit(X,Y)
print(modelo1.coef_, ' ', modelo1.intercept_)

[ 0.48074167  0.1743958   0.10445494  0.03651219 -0.12767306 -0.14642575
  0.0740882   0.04695625 -0.02627628  0.3309483  -0.08358169 -0.32214154
 -0.1123638  -0.20918346  0.3075537   0.12910636  0.19574074  0.0065652
  0.45644811  0.17687011  0.07576428 -0.34604682 -0.22271567  0.22592087
 -0.14446248 -0.27200823 -0.17330196  0.1296851  -0.13393978  0.0969944
 -0.10475355 -0.16053915  0.04587908  0.35687576  0.04700155  0.01200662
 -0.18501947  0.17677204  0.062995    0.18884682  0.95732993]   -1.1154880360082764


Bondad de Ajuste

In [72]:
R2 = modelo1.score(X,Y)
n, k = np.shape(X)
k = k + 1
R2a = 1 - (n-1)/(n-k)*(1-R2)
print(R2,' ',R2a)

0.8457652439207497   0.8278512920815166


Prediccion

In [73]:
Yp = modelo1.predict(X)
print('MSE', mean_squared_error(Y,Yp))

MSE 3.2291325973085345


Regresion Polinomica

In [91]:
for i in range(1,4):
  X_pol = PolynomialFeatures(i).fit_transform(Datos[var_cuant]) 
  X_pol = pd.DataFrame(X_pol, index = Dicos.index.values)
  X_pol = pd.concat([Dicos, X_pol], axis=1)
  m1 = reg.fit(X_pol, Y) 
  Yp = m1.predict(X_pol) 
  print(mean_squared_error(Y, Yp)) 

3.2291325973085345
1.844470617137806
5.261408102547894e-25


# Evaluación dentro y fuera de la muestra de entrenamiento


In [82]:
Xen, Xpr, Yen, Ypr = train_test_split(X, Y)
reg = LinearRegression()
reg.fit(Xen, Yen)
Yen_p = reg.predict(Xen) 
Ypr_p = reg.predict(Xpr) #Predicción con datos de prueba
#imprime los errores cuadráticos medios de entrenamiento y de prueba
print('MSE entrenamiento:', mean_squared_error(Yen_p, Yen), 'MSE prueba', mean_squared_error(Ypr_p, Ypr))


MSE entrenamiento: 3.4286272318325546 MSE prueba 3.176966960901041


# Regresión Ridge

In [85]:
lambdas = {'alpha': np.arange(1, 100)}
ridges = GridSearchCV(Ridge(), lambdas, scoring='neg_mean_squared_error', cv=5)
ridges.fit(X, Y)
print(ridges.best_params_)
print(ridges.best_score_) 

{'alpha': 99}
-3.819604517353196


# Regresión Lasso

In [86]:
alfas = {'alpha': np.arange(1, 100)}
lasos = GridSearchCV(Lasso(), alfas, scoring='neg_mean_squared_error', cv=5)
lasos.fit(X, Y)
print(lasos.best_params_)
print(lasos.best_score_)

{'alpha': 1}
-3.891875858383552


# Comparando Modelos

In [94]:

def modelos(X,Y): #Función denominada "modelos" que recibe parámetros X y Y
  reg = LinearRegression()
  mses = cross_val_score(reg, X, Y, scoring='neg_mean_squared_error', cv=5)
  lambdas = {'alpha': np.arange(1, 100)}
  rid = Ridge(normalize = True) #Establece que se estandaricen las variables del modelo
  ridges = GridSearchCV(rid, lambdas, scoring='neg_mean_squared_error', cv=5)
  ridges.fit(X, Y)
  las = Lasso(normalize = True)
  lasos = GridSearchCV(las, lambdas, scoring='neg_mean_squared_error', cv=5,)
  lasos.fit(X, Y)
  print('reg lineal: ', np.mean(mses))
  print('hiperparam ridge: ', ridges.best_params_)
  print('reg ridge: ',ridges.best_score_)
  print('hiperparam lasso: ', lasos.best_params_)
  print('reg lasso: ',lasos.best_score_)

In [97]:
for i in range(1,4):
  X_pol = PolynomialFeatures(i).fit_transform(Datos[var_cuant]) 
  X_pol = pd.DataFrame(X_pol, index = Dicos.index.values)
  X_pol = pd.concat([Dicos, X_pol], axis=1)
  print('Modelo polinomial para grado', i)
  modelos(X_pol, Y)


Modelo polinomial para grado 1
reg lineal:  -4.288584201094516
hiperparam ridge:  {'alpha': 1}
reg ridge:  -6.606779530650144
hiperparam lasso:  {'alpha': 1}
reg lasso:  -21.256092773593977
Modelo polinomial para grado 2
reg lineal:  -1.3239689819745407e+19
hiperparam ridge:  {'alpha': 1}
reg ridge:  -4.249476569920891
hiperparam lasso:  {'alpha': 1}
reg lasso:  -21.256092773593977
Modelo polinomial para grado 3
reg lineal:  -233.80694942182117
hiperparam ridge:  {'alpha': 1}
reg ridge:  -4.139744002577249
hiperparam lasso:  {'alpha': 1}
reg lasso:  -21.256092773593977


# Conclusion


Como conclusion podemos evidenciar que el MSE optimo es el que se encuentra en el modelo de Regresion Polinomial de grado 2 , utilizando variables cuantitativas y Categoricas(los cuales fueron transformadas a Dicotomicas), pora la validacion se utilizo una validacion cruzada.
para dicho analisis se utilizaron los modelos de regresion linea , regresion polinomial, Regresion de Ridge y Regresion de Lasso.
